IMPORT STATEMENTS

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import pybel
import os
import sys
import getopt


Prepping receptor & ligand from scratch using RDKIT

In [3]:
def load_and_prepare_molecule(filepath, remove_waters=True, is_receptor=False):
    """
    Load a molecule from a file, optionally remove waters, and prepare by adding hydrogens and optimizing.
    """
    print(f"Loading molecule from: {filepath}")
    if filepath.endswith('.pdb'):
        molecule = Chem.MolFromPDBFile(filepath, removeHs=False)
    elif filepath.endswith('.mol'):
        molecule = Chem.MolFromMolFile(filepath, removeHs=False)
    else:
        raise ValueError("Unsupported file format")

    if is_receptor and remove_waters:
        print("Removing water molecules from receptor...")
        new_molecule = Chem.RWMol(molecule)
        for atom in reversed(new_molecule.GetAtoms()):
            if atom.GetPDBResidueInfo().GetResidueName() == 'HOH':
                new_molecule.RemoveAtom(atom.GetIdx())
        molecule = new_molecule.GetMol()

    print("Adding hydrogens and optimizing molecule...")
    molecule = Chem.AddHs(molecule)
    # print('Computing molecular geometry...')
    # AllChem.EmbedMolecule(molecule)
    # print('Computing force field...')
    # AllChem.MMFFOptimizeMolecule(molecule)
    print("Computing Gasteiger charges...")
    AllChem.ComputeGasteigerCharges(molecule)
    return molecule

def convert_to_pdbqt(molecule, output_filename):
    """
    Convert an RDKit molecule to a PDBQT file using Pybel.
    """
    print(f"Converting molecule to PDBQT and saving to {output_filename}")
    molblock = Chem.MolToMolBlock(molecule)
    pybel_mol = pybel.readstring("mol", molblock)
    # pybel_mol.calccharges(method='gasteiger')
    pybel_mol.write('pdbqt', output_filename, overwrite=True)

def prepare_receptor(filepath):
    """
    Prepare the receptor: load, prepare structure, and save as PDBQT.
    """
    print(f"Preparing receptor from file: {filepath}")
    receptor = load_and_prepare_molecule(filepath, remove_waters=True, is_receptor=True)
    receptor_pdbqt = filepath.replace('.pdb', '_receptor.pdbqt')
    convert_to_pdbqt(receptor, receptor_pdbqt)
    return receptor_pdbqt

def prepare_ligand(filepath):
    """
    Prepare the ligand: load, prepare structure, and save as PDBQT.
    """
    print(f"Preparing ligand from file: {filepath}")
    ligand = load_and_prepare_molecule(filepath, remove_waters=False, is_receptor=False)
    ligand_pdbqt = filepath.replace('.pdb', '_ligand.pdbqt')
    convert_to_pdbqt(ligand, ligand_pdbqt)
    return ligand_pdbqt

In [ ]:
receptor_pdbqt = prepare_receptor('or51e2unprep.pdb')
ligand_pdbqt = prepare_ligand('propionate.pdb')

print("Receptor prepared as:", receptor_pdbqt)
print("Ligand prepared as:", ligand_pdbqt)

Using vinadocking github module to prepare receptor
Only able to prepare by adding charges

In [ ]:
def main():
    # process command arguments
    try:
        opt_list, args = getopt.getopt(sys.argv[1:], 'r:vo:A:Cp:U:eM:d:')
    except getopt.GetoptError as e:
        print(f'prepare_receptor4.py: {e}')
        usage()
        sys.exit(2)

    # initialize required parameters
    receptor_filename = "or51e2unprep.pdb"

    # optional parameters
    verbose = None
    repairs = ''
    charges_to_add = 'gasteiger'
    preserve_charge_types = None
    cleanup = "nphs_lps_waters_nonstdres"
    outputfilename = "prepreceptor.pdbqt"
    mode = 'automatic'
    delete_single_nonstd_residues = None
    dictionary = None

    for o, a in opt_list:
        if o in ('-r', '--r'):
            receptor_filename = a
            if verbose:
                print(f'set receptor_filename to {a}')
        if o in ('-v', '--v'):
            verbose = True
            if verbose:
                print('set verbose to True')
        if o in ('-o', '--o'):
            outputfilename = a
            if verbose:
                print(f'set outputfilename to {a}')
        if o in ('-A', '--A'):
            repairs = a
            if verbose:
                print(f'set repairs to {a}')
        if o in ('-C', '--C'):
            charges_to_add = None
            if verbose:
                print('do not add charges')
        if o in ('-p', '--p'):
            if not preserve_charge_types:
                preserve_charge_types = a
            else:
                preserve_charge_types = f'{preserve_charge_types},{a}'
            if verbose:
                print(f'preserve initial charges on {preserve_charge_types}')
        if o in ('-U', '--U'):
            cleanup = a
            if verbose:
                print(f'set cleanup to {a}')
        if o in ('-e', '--e'):
            delete_single_nonstd_residues = True
            if verbose:
                print('set delete_single_nonstd_residues to True')
        if o in ('-M', '--M'):
            mode = a
            if verbose:
                print(f'set mode to {a}')
        if o in ('-d', '--d'):
            dictionary = a
            if verbose:
                print(f'set dictionary to {dictionary}')
        if o in ('-h', '--'):
            usage()
            sys.exit()

    if not receptor_filename:
        print('prepare_receptor4: receptor filename must be specified.')
        usage()
        sys.exit()

    # Reading the molecule using RDKit
    mol = None
    if receptor_filename.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(receptor_filename)
    elif receptor_filename.endswith('.mol2'):
        mol = Chem.MolFromMol2File(receptor_filename)
    elif receptor_filename.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(receptor_filename)
        mol = next(supplier) if supplier else None
    else:
        print(f'Unsupported file format for {receptor_filename}')
        sys.exit(1)
    
    if not mol:
        print(f'Could not read molecule from {receptor_filename}')
        sys.exit(1)
    
    if verbose:
        print(f'Read {receptor_filename}')
    
    if 'bonds' in repairs:
        # Build bonds by distance (RDKit usually does this automatically)
        if verbose:
            print('Building bonds')
        AllChem.EmbedMolecule(mol)

    if 'hydrogens' in repairs:
        # Add hydrogens
        if verbose:
            print('Adding hydrogens')
        mol = Chem.AddHs(mol)

    if charges_to_add == 'gasteiger':
        if verbose:
            print('Adding Gasteiger charges')
        AllChem.ComputeGasteigerCharges(mol)

    # Handle output filename
    if not outputfilename:
        outputfilename = os.path.splitext(os.path.basename(receptor_filename))[0] + '.pdbqt'
    
    # Write the prepared receptor to output file (using pdbqt format)
    with open(outputfilename, 'w') as f:
        f.write(Chem.MolToPDBBlock(mol))
        if verbose:
            print(f'Wrote prepared receptor to {outputfilename}')

if __name__ == '__main__':
    main()

# To execute this command type:
# prepare_receptor4.py -r pdb_file -o outputfilename -A checkhydrogens 